# PASAR PARAMETROS Y PONER UNA CELDA NEUVA CON LOS VALORES PASADOS COMO PARAMETROS

In [1]:
import os
import papermill as pm
from IPython.display import display, HTML

def procesar_notebooks(notebooks, directorio_notebooks_procesados="./notebooks_procesados"):
    """
    Ejecuta una lista de notebooks de Jupyter con parámetros especificados y
    guarda los resultados en el directorio de notebooks procesados.

    Parámetros:
    notebooks (list): Una lista de tuplas con el nombre del notebook, la ruta de salida y los parámetros a pasar.
    directorio_notebooks_procesados (str): El directorio donde se guardarán los notebooks procesados.
    """
    # Asegurarse de que el directorio de notebooks procesados exista
    if not os.path.exists(directorio_notebooks_procesados):
        os.makedirs(directorio_notebooks_procesados)  # Crea el directorio si no existe
        print(f"Directorio {directorio_notebooks_procesados} creado.")

    # Ejecutar cada notebook con sus parámetros específicos
    for input_path, output_path, params in notebooks:
        print(f"Ejecutando {input_path} con parámetros {params}...")
        pm.execute_notebook(
            input_path=input_path,
            output_path=output_path,
            parameters=params
        )
        # Crear el hipervínculo
        display(HTML(f"Ejecutado correctamente, verifique el resultado en el siguiente link -> <a href='{output_path}' target='_blank'>{output_path}</a>"))
        print()
        
directorio_procesados = "./notebooks_procesados"
# Lista de notebooks con parámetros específicos
notebooks_con_parametros = [
    ("1_sumar.ipynb", f"{directorio_procesados}/sumar_ejecutado.ipynb", {"x": 10, "y": 5}),
    ("2_restar.ipynb", f"{directorio_procesados}/restar_ejecutado.ipynb", {"x": 10, "y": 3}),
    ("3_multiplicar.ipynb",f"{directorio_procesados}/multiplicar_ejecutado.ipynb", {"x": 4, "y": 7}),
    ("4_dividir.ipynb", f"{directorio_procesados}/dividir_ejecutado.ipynb", {"x": 20, "y": 5}),
]

# Llamar a la función para ejecutar los notebooks
procesar_notebooks(notebooks_con_parametros)

Directorio ./notebooks_procesados creado.
Ejecutando 1_sumar.ipynb con parámetros {'x': 10, 'y': 5}...


Executing:   0%|          | 0/4 [00:00<?, ?cell/s]


Ejecutando 2_restar.ipynb con parámetros {'x': 10, 'y': 3}...


Executing:   0%|          | 0/4 [00:00<?, ?cell/s]


Ejecutando 3_multiplicar.ipynb con parámetros {'x': 4, 'y': 7}...


Executing:   0%|          | 0/4 [00:00<?, ?cell/s]


Ejecutando 4_dividir.ipynb con parámetros {'x': 20, 'y': 5}...


Executing:   0%|          | 0/4 [00:00<?, ?cell/s]

# PASAR PARAMETROS Y ELIMINAR LA CELDA DONDE SE ESPERABAN LOS PARAMETROS Y ADICIONAR LA CELDA CON LOS PARAMETROS ADICONADOS

In [2]:
import json
import papermill as pm
from IPython.display import display, HTML
import os

def reemplazar_parametros_en_nuevo_notebook(notebook_path, nuevos_parametros, output_path):
    """
    Lee el notebook original, reemplaza los parámetros y guarda un nuevo notebook procesado sin la celda de parámetros.
    
    Parámetros:
    notebook_path (str): Ruta al archivo original del notebook.
    nuevos_parametros (dict): Diccionario con los nuevos valores de los parámetros a reemplazar.
    output_path (str): Ruta donde se guardará el nuevo notebook con los parámetros reemplazados.
    """
    # Leer el notebook original
    with open(notebook_path, 'r') as f:
        notebook = json.load(f)
    
    # Crear una copia del notebook original sin modificarlo
    nuevo_notebook = notebook.copy()
    
    # Reemplazar los parámetros en la copia
    for cell in nuevo_notebook['cells']:
        if 'metadata' in cell and 'tags' in cell['metadata'] and 'parameters' in cell['metadata']['tags']:
            # Reemplazar los valores de los parámetros
            cell['source'] = "\n".join([f"{k} = {v}" for k, v in nuevos_parametros.items()])

    # Guardar el nuevo notebook con los parámetros reemplazados (sin tocar el original)
    with open(output_path, 'w') as f:
        json.dump(nuevo_notebook, f, indent=4)

def procesar_notebooks(notebooks, directorio_notebooks_procesados="./notebooks_procesados"):
    """
    Ejecuta una lista de notebooks de Jupyter con parámetros especificados y
    guarda los resultados en el directorio de notebooks procesados.
    
    Parámetros:
    notebooks (list): Una lista de tuplas con el nombre del notebook, la ruta de salida y los parámetros a pasar.
    directorio_notebooks_procesados (str): El directorio donde se guardarán los notebooks procesados.
    """
    # Asegurarse de que el directorio de notebooks procesados exista
    if not os.path.exists(directorio_notebooks_procesados):
        os.makedirs(directorio_notebooks_procesados)  # Crea el directorio si no existe
        print(f"Directorio {directorio_notebooks_procesados} creado.")

    # Ejecutar cada notebook con sus parámetros específicos
    for input_path, output_path, params in notebooks:
        # Crear un nuevo notebook procesado con los parámetros reemplazados
        print(f"Procesando el notebook {input_path} con parámetros {params}...")
        
        # Crear el nuevo notebook con los parámetros reemplazados
        reemplazar_parametros_en_nuevo_notebook(input_path, params, output_path)
        
        # El notebook procesado ya está guardado, por lo que no es necesario abrirlo nuevamente.
        # Ahora lo ejecutamos con papermill
        print(f"Ejecutando {output_path}...")
        pm.execute_notebook(
            input_path=output_path,  # Usamos el notebook procesado con parámetros reemplazados
            output_path=output_path,
            parameters=params
        )
        
        # Después de la ejecución, eliminar la celda de parámetros (que ya no es necesaria)
        with open(output_path, 'r') as f:
            nuevo_notebook = json.load(f)

        # Eliminar la celda de parámetros (ya no se necesita)
        for i, cell in enumerate(nuevo_notebook['cells']):
            if 'metadata' in cell and 'tags' in cell['metadata'] and 'parameters' in cell['metadata']['tags']:
                del nuevo_notebook['cells'][i]
                break

        # Guardar el notebook sin la celda de parámetros (ya está completamente guardado)
        with open(output_path, 'w') as f:
            json.dump(nuevo_notebook, f, indent=4)

        # Crear el hipervínculo para verificar el resultado
        display(HTML(f"Ejecutado correctamente, verifique el resultado en el siguiente link -> <a href='{output_path}' target='_blank'>{output_path}</a>"))
        print()

# Directorio donde se almacenarán los notebooks procesados
directorio_procesados = "./notebooks_procesados"

# Lista de notebooks con parámetros específicos
notebooks_con_parametros = [
    ("1_sumar.ipynb", f"{directorio_procesados}/sumar_ejecutado.ipynb", {"x": 10, "y": 5}),
    ("2_restar.ipynb", f"{directorio_procesados}/restar_ejecutado.ipynb", {"x": 10, "y": 3}),
    ("3_multiplicar.ipynb", f"{directorio_procesados}/multiplicar_ejecutado.ipynb", {"x": 4, "y": 7}),
    ("4_dividir.ipynb", f"{directorio_procesados}/dividir_ejecutado.ipynb", {"x": 20, "y": 5}),
]

# Llamar a la función para ejecutar los notebooks
procesar_notebooks(notebooks_con_parametros)

Procesando el notebook 1_sumar.ipynb con parámetros {'x': 10, 'y': 5}...
Ejecutando ./notebooks_procesados/sumar_ejecutado.ipynb...


Executing:   0%|          | 0/4 [00:00<?, ?cell/s]


Procesando el notebook 2_restar.ipynb con parámetros {'x': 10, 'y': 3}...
Ejecutando ./notebooks_procesados/restar_ejecutado.ipynb...


Executing:   0%|          | 0/4 [00:00<?, ?cell/s]


Procesando el notebook 3_multiplicar.ipynb con parámetros {'x': 4, 'y': 7}...
Ejecutando ./notebooks_procesados/multiplicar_ejecutado.ipynb...


Executing:   0%|          | 0/4 [00:00<?, ?cell/s]


Procesando el notebook 4_dividir.ipynb con parámetros {'x': 20, 'y': 5}...
Ejecutando ./notebooks_procesados/dividir_ejecutado.ipynb...


Executing:   0%|          | 0/4 [00:00<?, ?cell/s]